In [ ]:
import os, re
import contextlib
import tempfile
from gvxrPython3 import gvxr

In [ ]:
# From https://stackoverflow.com/questions/11265603/how-do-i-export-the-output-of-pythons-built-in-help-function

def write_help(fp, func):
    with contextlib.redirect_stdout(fp):
        help(func)

In [ ]:
call_dictionary = {}

# For each file in the directory
for fname in os.listdir():

    # If the file is a Python notebook
    if fname[-6:] == ".ipynb":

        # Open the file
        with open(fname, "r") as f:

            # Initialisation
            cell_id = 0;
            
            # Read each line
            for line_id, line in enumerate(f.readlines()):

                if """cell_type": "code""" in line:
                    cell_id += 1;

                # The line includes a call to a function from gVXR
                if "gvxr." in line and "`" not in line:

                    # Where does the call start
                    start_index = line.find("gvxr.");

                    # Where might the function name end
                    stop_indices = [_.start() for _ in re.finditer("\(", line[start_index:])];

                    # This is is function call
                    if len(stop_indices) > 0:

                        # Find the right stop index
                        stop_index = None;
                        for i in stop_indices:
                            if i > start_index:
                                stop_index = i;
                                break;

                        # stop_index is valid
                        if stop_index:
                            function_name = line[start_index:start_index+stop_index]

                            # Add the function name to the dictionary if needed
                            if function_name not in call_dictionary:
                                call_dictionary[function_name] = {};

                            # Add the file name to the dictionary and create an empty array if needed
                            if fname not in call_dictionary[function_name]:
                                call_dictionary[function_name][fname] = [];

                            # Add the line number (maybe it should be the cell number)
                            # call_dictionary[function_name][fname].append(line_id + 1);

                            # Add the cell number if needed
                            if cell_id not in call_dictionary[function_name][fname]:
                                call_dictionary[function_name][fname].append(cell_id);


In [ ]:
# Create a temporary file
fp = tempfile.NamedTemporaryFile(mode="w", delete=False)

# Fetch the help string for each function called
for function_name_key in call_dictionary.keys():
    write_help(fp, "gvxrPython3." + function_name_key)

# Close a delete the temporary file
fp.close()

In [ ]:
# Open the file
with open(fp.name, "r") as f:

    current_id0 = -1
    current_id1 = -1
    current_id2 = -1
    
    # Read each line
    for line_id, line in enumerate(f.readlines()):

        if "Help on function" in line:
            start_id = len("Help on function") + 1;
            end_id = line.find("in gvxrPython3.gvxr:") - 1
            function_name = line[start_id:end_id];

            current_id0 = line_id;
        elif current_id0 + 1 == line_id: # and len(line) == 1:
            current_id1 = line_id;
        elif current_id0 + 2 == line_id and current_id1 + 1 == line_id and "gvxrPython3.gvxr." in line:
            current_id2 = line_id;
        elif current_id0 + 3 == line_id and current_id1 + 2 == line_id and current_id2 + 1 == line_id:
            help_string = line[4:len(line)-1];
            call_dictionary["gvxr." + function_name]["help"] = help_string            
        else:
            current_id0 = -1
            current_id1 = -1
            current_id2 = -1

In [ ]:
# Dlete a delete the temporary file
os.remove(fp.name) 

In [ ]:
print("Glossary of gVXR's function calls")
for function_name_key in call_dictionary.keys():
    # print(function_name_key)
    print("- `" + function_name_key + "`:")

    if "help" in call_dictionary[function_name_key]:
        description = call_dictionary[function_name_key]["help"];

        if len(description):
            print("  - Description:", description);
        else:    
            print("  - Description: ***** MISSING *****");
    else:    
        print("  - Description: +++++ MISSING +++++");
 
    for fname in call_dictionary[function_name_key].keys():
        if fname != "help":
            print("  - Called in file:", fname);

            if len(call_dictionary[function_name_key][fname]) == 1:
                print("    - At Cell:", call_dictionary[function_name_key][fname]);
            else: 
                print("    - At Cells:", call_dictionary[function_name_key][fname]);
            print();

    print();